In [10]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
import pandas as pd
import dice_ml
import torch
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tqdm import tqdm


In [11]:
data, labels = datasets.make_moons(n_samples=1000, shuffle=True, noise=0.2, random_state=None)
dic = {'x':data[:,0], 'y' : data[:,1], 'target': labels}
df = pd.DataFrame(data = dic)


In [12]:
data_dice = dice_ml.Data(dataframe=df, continuous_features=['x','y'], outcome_name='target')

In [13]:
X = df[['x','y']].values
y = df[['target']].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
dic_train = {'x':x_train[:,0], 'y' : x_train[:,1], 'target': y_train.flatten()}
df_train = pd.DataFrame(data = dic_train)
dic_test = {'x':x_test[:,0], 'y' : x_test[:,1], 'target': y_test.flatten()}
df_test = pd.DataFrame(data = dic_test)
data_dice_test = dice_ml.Data(dataframe=df_test, continuous_features=['x','y'], outcome_name='target')

In [14]:
def plot_decision_boundary(pred_func,X,y,title, obs, enemy):
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    # Set min and max values and give it some padding
    h = 0.01
    # Generate a grid of points with distance h between them
    xx,yy=np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.binary)
    plt.scatter(obs[0], obs[1], c = 'lime',marker= 'x')
    plt.scatter(enemy.T[0], enemy.T[1], c = 'red', marker = 'o')
    plt.title(title)
    plt.show()

In [15]:
def grad(x_train, y_train, model, critetion, epochs=100, learning_rate=0.01):
    #print(x_train)
    #print(y_train)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  
    for epoch in range(epochs):
        # Converting inputs and labels to Variable
        if torch.cuda.is_available():
            inputs = Variable(torch.from_numpy(x_train.astype(np.float32)).cuda())
            labels = Variable(torch.from_numpy(y_train.astype(np.float32)).cuda())
        else:
            inputs = Variable(torch.from_numpy(x_train.astype(np.float32)))
            labels = Variable(torch.from_numpy(y_train.astype(np.float32)))

        # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
        optimizer.zero_grad()
        # get output from the model, given the inputs
        outputs = model(inputs)
        # geloss for the predicted output
        #print(outputs)
        loss = criterion(outputs, labels)
        # get gradients w.r.t to parameters
        loss.backward()
        # update parameters
        optimizer.step()

def predict(x, model):
    #Convert into numpy element to tensor
    x = torch.from_numpy(x).type(torch.FloatTensor)
    #Predict and return ans
    ans = model.predict(x)
    return ans.numpy()

In [16]:
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)
        self.activation = torch.nn.ReLU()
        

    def forward(self, x):
        #out = self.linear(x)
        out = self.activation(self.linear(x))
        return out
    
    #This function takes an input and predicts the class, (0 or 1)        
    def predict(self,x):
        #Apply softmax to output. 
        pred = self.forward(x)
        ans = []
        #Pick the class with maximum weight
        for t in pred:
            if t[0]>0.5:
                ans.append(0)
            else:
                ans.append(1)
        return torch.tensor(ans)

In [17]:
class res2D(torch.nn.Module):
    def __init__(self, inputSize, hiddenSize,outputSize):
        super().__init__()
        self.linear1 = torch.nn.Linear(inputSize, hiddenSize)
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(hiddenSize, outputSize)
        self.activation2 = torch.nn.Sigmoid()        

    def forward(self, x):
        out = self.activation2(self.linear2(self.activation1(self.linear1(x))))
        return out
    
    #This function takes an input and predicts the class, (0 or 1)        
    def predict(self,x):
        #Apply softmax to output. 
        pred = self.forward(x)
        ans = []
        #Pick the class with maximum weight
        for t in pred:
            if t[0]>0.5:
                ans.append(0)
            else:
                ans.append(1)
        return torch.tensor(ans)

In [18]:
model = linearRegression(2,1)
criterion = torch.nn.MSELoss()

grad(x_train, y_train, model, criterion)
model_dice = dice_ml.Model(model=model, backend='PYT')
explainer = dice_ml.Dice(data_dice, model_dice)
index=67
query_instance = dict(zip(['x','y'], x_test[index]))
counterfactuals = explainer.generate_counterfactuals(
                    query_instance=query_instance,total_CFs=1, proximity_weight=0.9,desired_class='opposite')
counterfactuals.visualize_as_dataframe()
plot_decision_boundary(lambda x : predict(x,model), x_test, y_test.flatten(), "",x_test[index], counterfactuals.cf_examples_list[0].final_cfs_df.values[0][:2] )

/home/manuel/.local/lib/python3.10/site-packages/dice_ml/data_interfaces/public_data_interface.py:582: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = self.ohe_base_df.append(query_instance, ignore_index=True, sort=False)


No Counterfactuals found for the given configuation,  perhaps try with different values of proximity (or diversity) weights or learning rate... ; total time taken: 00 min 09 sec
Query instance (original outcome : 0)


/home/manuel/.local/lib/python3.10/site-packages/dice_ml/data_interfaces/public_data_interface.py:582: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = self.ohe_base_df.append(query_instance, ignore_index=True, sort=False)
/home/manuel/.local/lib/python3.10/site-packages/dice_ml/data_interfaces/public_data_interface.py:582: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp = self.ohe_base_df.append(query_instance, ignore_index=True, sort=False)


,x,y,target
0,0.639692,-0.236087,0.0



No counterfactuals found!


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
model = res2D(2,20,1)
criterion = torch.nn.BCELoss()

grad(x_train, y_train, model, criterion, epochs=1000)
model_dice = dice_ml.Model(model=model, backend='PYT')
explainer = dice_ml.Dice(data_dice, model_dice)
index=67
query_instance = dict(zip(['x','y'], x_test[index]))
counterfactuals = explainer.generate_counterfactuals(
                    query_instance=query_instance,total_CFs=1, desired_class='opposite')
counterfactuals.visualize_as_dataframe()
plot_decision_boundary(lambda x : predict(x,model), x_test, y_test.flatten(), "",x_test[index], counterfactuals.cf_examples_list[0].final_cfs_df.values[0][:2] )

In [ ]:
model = linearRegression(2,1)
criterion = torch.nn.MSELoss()

grad(x_train, y_train, model, criterion)
model_dice = dice_ml.Model(model=model, backend='PYT')
explainer = dice_ml.Dice(data_dice, model_dice)
index=74
query_instance = dict(zip(['x','y'], x_test[index]))
proximity_weights = np.linspace(0.1, 10, num=15)
pred_ori = counterfactuals.cf_examples_list[0].test_pred
for proximity_weight in tqdm(proximity_weights):
    counterfactuals = explainer.generate_counterfactuals(
                        query_instance=query_instance,total_CFs=1, proximity_weight=proximity_weight,desired_class='opposite')
    print(proximity_weight)
    
    counterfactuals.visualize_as_dataframe()
    if len(counterfactuals.cf_examples_list[0].final_cfs_df.values) >0 :
        plot_decision_boundary(lambda x : predict(x,model), x_test, y_test.flatten(), "Proximity_weight"+str(proximity_weight),x_test[index], counterfactuals.cf_examples_list[0].final_cfs_df.values[0][:2] )

In [ ]:
model = linearRegression(2,1)
criterion = torch.nn.MSELoss()

grad(x_train, y_train, model, criterion)
model_dice = dice_ml.Model(model=model, backend='PYT')
explainer = dice_ml.Dice(data_dice, model_dice)
index=66
query_instance = dict(zip(['x','y'], x_test[index]))
proximity_weights = np.linspace(0.1, 8, num=15)
pred_ori = counterfactuals.cf_examples_list[0].test_pred
for proximity_weight in tqdm(proximity_weights):
    counterfactuals = explainer.generate_counterfactuals(
                        query_instance=query_instance,total_CFs=1, proximity_weight=proximity_weight,desired_class='opposite')
    print(proximity_weight)
    
    counterfactuals.visualize_as_dataframe()
    if len(counterfactuals.cf_examples_list[0].final_cfs_df.values) >0 :
        plot_decision_boundary(lambda x : predict(x,model), x_test, y_test.flatten(), "Proximity_weight"+str(proximity_weight),x_test[index], counterfactuals.cf_examples_list[0].final_cfs_df.values[0][:2] )

In [ ]:
model = linearRegression(2,1)
criterion = torch.nn.MSELoss()

grad(x_train, y_train, model, criterion)
model_dice = dice_ml.Model(model=model, backend='PYT')
explainer = dice_ml.Dice(data_dice, model_dice)
index=74
query_instance = dict(zip(['x','y'], x_test[index]))
diversity_weights = np.linspace(0.1, 10, num=15)
pred_ori = counterfactuals.cf_examples_list[0].test_pred
for diversity_weight in tqdm(diversity_weights):
    counterfactuals = explainer.generate_counterfactuals(query_instance=query_instance,total_CFs=1, diversity_weight=diversity_weight,desired_class='opposite')
    print(diversity_weight)
    
    counterfactuals.visualize_as_dataframe()
    if len(counterfactuals.cf_examples_list[0].final_cfs_df.values) >0 :
        plot_decision_boundary(lambda x : predict(x,model), x_test, y_test.flatten(), "diversity weight "+str(diversity_weight),x_test[index], counterfactuals.cf_examples_list[0].final_cfs_df.values[0][:2] )

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
breast_cancer = datasets.load_breast_cancer(as_frame= True)



data = breast_cancer['data']
target = breast_cancer['target']

feature_names = breast_cancer['feature_names']
target_names = breast_cancer['target_names']

X_train, X_test, y_train, y_test = train_test_split(data, target, train_size=0.75, random_state = 1234)

In [ ]:
import dice_ml
data['target'] = target
data_dice = dice_ml.Data(dataframe=data, continuous_features = list(feature_names), outcome_name='target')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
obs2 = X_test.iloc[[0]]

rfClassifier = RandomForestClassifier()
fittedRF = rfClassifier.fit(X_train, y_train)
prediction = rfClassifier.predict(X_test)
# Using sklearn backend
m = dice_ml.Model(model= fittedRF, backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(data_dice, m, method= 'random')


In [ ]:
counterfactuals = exp.generate_counterfactuals(
                    query_instances=obs2, total_CFs=20, proximity_weight=0.1,desired_class='opposite')
counterfactuals.visualize_as_dataframe()

In [ ]:


def calc_distances(cfs, obs):
    cfmatrix = cfs.cf_examples_list[0].final_cfs_df.values[:,:-1]
    N = len(cfmatrix)
    print(N)
    dist = np.zeros_like(obs)
    best_norm = np.inf
    for cf in cfmatrix:
        dist = ((obs - cf )/ N)
        print(dist)
        norm = np.linalg.norm( obs - cf)
        if norm < best_norm:
            best_norm = norm
            best_enemy = cf
    return best_enemy, dist

In [ ]:
counterfactuals.cf_examples_list[0].final_cfs_df.drop("target", axis = 1)
best, distCF = calc_distances(counterfactuals, obs2.values)


In [ ]:
obsclass = y_test.iloc[[choice]].values[0]
print(obs2.values)

In [ ]:

ylabeldata(ind, obs2.values[0], feature_names, "Observation, class = " + str(target_names[obsclass]))

In [ ]:
predenem = fittedRF.predict(best.reshape(1, -1))
print(predenem)
ylabeldata(ind, best, feature_names, "Best Counterfactual, class = " + str(target_names[predenem]))

In [ ]:
print(distCF)

In [ ]:
ylabeldata(ind, np.abs(distCF[0]/20), feature_names, "Mean distances to counterfactuals")

In [ ]:
ylabeldata(ind, np.abs(obs2.values[0]- best), feature_names, "Distance Observation - counterfactual")